In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, when, desc, lit,
    count as spark_count,
    min as spark_min,
    max as spark_max
)
from pyspark.sql.types import *
import pandas as pd

StatementMeta(, 9d1c3a59-d18e-4fdf-8498-c7eee8f1135d, 3, Finished, Available, Finished)

In [2]:
spark = SparkSession.builder.appName("Bronze_Exploration").getOrCreate()

StatementMeta(, 9d1c3a59-d18e-4fdf-8498-c7eee8f1135d, 4, Finished, Available, Finished)

In [3]:
# Lista de tablas a analizar
tables = [
    "brz_redata_balance_balance_electrico",
    "brz_redata_demanda_evolucion",
    "brz_redata_generacion_estructura_generacion",
    "brz_redata_generacion_estructura_generacion_emisiones_asociadas",
    "brz_redata_generacion_estructura_renovables",
    "brz_redata_generacion_evolucion_renovable_no_renovable",
    "brz_redata_generacion_maxima_renovable",
    "brz_redata_mercados_componentes_precio"
]

StatementMeta(, 9d1c3a59-d18e-4fdf-8498-c7eee8f1135d, 5, Finished, Available, Finished)

In [4]:
# ============================================
# 1. RESUMEN GENERAL DE TODAS LAS TABLAS
# ============================================
print("\n📊 1. RESUMEN GENERAL")
print("-" * 80)

summary_data = []
for table in tables:
    df = spark.table(table)
    row_count = df.count()
    cols = len(df.columns)
    min_date = df.agg(spark_min("datetime")).collect()[0][0]
    max_date = df.agg(spark_max("datetime")).collect()[0][0]
    
    summary_data.append({
        "tabla": table.replace("brz_redata_", ""),
        "registros": f"{row_count:,}",
        "columnas": cols,
        "fecha_min": str(min_date)[:10] if min_date else "N/A",
        "fecha_max": str(max_date)[:10] if max_date else "N/A"
    })
    
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

StatementMeta(, 9d1c3a59-d18e-4fdf-8498-c7eee8f1135d, 6, Finished, Available, Finished)


📊 1. RESUMEN GENERAL
--------------------------------------------------------------------------------
                                               tabla registros  columnas  fecha_min  fecha_max
                           balance_balance_electrico    74,858        14 2023-09-30 2025-09-30
                                   demanda_evolucion     6,381        14 2023-09-30 2025-09-30
                    generacion_estructura_generacion    49,653        14 2023-09-30 2025-09-30
generacion_estructura_generacion_emisiones_asociadas    43,028        14 2023-09-30 2025-09-30
                    generacion_estructura_renovables    27,008        14 2023-09-30 2025-09-30
         generacion_evolucion_renovable_no_renovable    11,935        14 2023-09-30 2025-09-30
                         generacion_maxima_renovable       173        14 2023-09-30 2025-09-30
                         mercados_componentes_precio       156        14 2023-09-30 2025-09-30


In [5]:
# ============================================
# 2. ANÁLISIS DETALLADO POR TABLA
# ============================================
print("\n" + "=" * 80)
print("📋 2. ANÁLISIS DETALLADO POR TABLA")
print("=" * 80)

for table in tables:
    print(f"\n{'─' * 80}")
    print(f"📦 {table}")
    print(f"{'─' * 80}")
    
    df = spark.table(table)
    
    # Schema
    print("\n🔹 Schema:")
    df.printSchema()
    
    # Conteos por dimensiones clave
    print("\n🔹 Distribución por geo_limit:")
    df.groupBy("geo_limit").agg(spark_count("*").alias("count")).orderBy(desc("count")).show(10, truncate=False)
    
    print("🔹 Distribución por time_trunc:")
    df.groupBy("time_trunc").agg(spark_count("*").alias("count")).orderBy(desc("count")).show(10, truncate=False)
    
    # Valores únicos en columnas clave
    print(f"🔹 Valores únicos:")
    print(f"  - geo_id: {df.select('geo_id').distinct().count()}")
    print(f"  - geo_name: {df.select('geo_name').distinct().count()}")
    print(f"  - series_type: {df.select('series_type').distinct().count()}")
    print(f"  - metric_type: {df.select('metric_type').distinct().count()}")
    
    # Nulls
    print("\n🔹 Análisis de nulos:")
    null_counts = df.select([
        spark_count(when(col(c).isNull(), c)).alias(c) 
        for c in df.columns
    ]).collect()[0].asDict()
    
    nulls_found = {k: v for k, v in null_counts.items() if v > 0}
    if nulls_found:
        for col_name, null_count in nulls_found.items():
            pct = (null_count / df.count()) * 100
            print(f"  - {col_name}: {null_count:,} ({pct:.2f}%)")
    else:
        print("  ✅ Sin nulos en columnas principales")
    
    # Muestra de series disponibles
    print("\n🔹 Series disponibles (top 5):")
    df.select("series_title", "metric_title") \
      .distinct() \
      .limit(5) \
      .show(5, truncate=False)
    
    # Rango temporal detallado
    print("🔹 Cobertura temporal:")
    temporal = df.groupBy("time_trunc") \
                 .agg(
                     spark_min("datetime").alias("fecha_inicio"),
                     spark_max("datetime").alias("fecha_fin"),
                     spark_count("*").alias("registros")
                 ) \
                 .orderBy("time_trunc")
    temporal.show(truncate=False)
    
    # Detectar posibles duplicados
    print("🔹 Verificación de duplicados:")
    total = df.count()
    unique = df.dropDuplicates(["geo_id", "time_trunc", "series_type", "datetime"]).count()
    if total == unique:
        print(f"  ✅ Sin duplicados ({total:,} registros únicos)")
    else:
        print(f"  ⚠️ Posibles duplicados: {total - unique:,} registros")

StatementMeta(, 9d1c3a59-d18e-4fdf-8498-c7eee8f1135d, 7, Finished, Available, Finished)


📋 2. ANÁLISIS DETALLADO POR TABLA

────────────────────────────────────────────────────────────────────────────────
📦 brz_redata_balance_balance_electrico
────────────────────────────────────────────────────────────────────────────────

🔹 Schema:
root
 |-- geo_id: long (nullable = true)
 |-- geo_name: string (nullable = true)
 |-- geo_limit: string (nullable = true)
 |-- ccaa_name: string (nullable = true)
 |-- time_trunc: string (nullable = true)
 |-- series_type: string (nullable = true)
 |-- series_title: string (nullable = true)
 |-- metric_type: string (nullable = true)
 |-- metric_title: string (nullable = true)
 |-- is_composite: boolean (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- value: double (nullable = true)
 |-- percentage: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)


🔹 Distribución por geo_limit:
+----------+-----+
|geo_limit |count|
+----------+-----+
|peninsular|24278|
|baleares  |16084|
|canarias  |13627|
|ccaa 

In [6]:
# ============================================
# 3. ANÁLISIS DE VALORES Y RANGOS
# ============================================
print("\n" + "=" * 80)
print("📈 3. ANÁLISIS DE VALORES")
print("=" * 80)

for table in tables:
    print(f"\n{'─' * 80}")
    print(f"📦 {table}")
    print(f"{'─' * 80}")
    
    df = spark.table(table)
    
    # Stats de valores
    print("\n🔹 Estadísticas de 'value':")
    df.select("value").summary("count", "mean", "stddev", "min", "max").show(truncate=False)
    
    # Porcentajes (si existen)
    if "percentage" in df.columns:
        non_null_pct = df.filter(col("percentage").isNotNull()).count()
        if non_null_pct > 0:
            print("🔹 Estadísticas de 'percentage':")
            df.filter(col("percentage").isNotNull()) \
              .select("percentage") \
              .summary("count", "mean", "min", "max") \
              .show(truncate=False)
    
    # Valores extremos o anómalos
    print("🔹 Top 5 valores máximos:")
    df.orderBy(desc("value")) \
      .select("datetime", "geo_name", "series_title", "value") \
      .limit(5) \
      .show(5, truncate=False)

StatementMeta(, 9d1c3a59-d18e-4fdf-8498-c7eee8f1135d, 8, Finished, Available, Finished)


📈 3. ANÁLISIS DE VALORES

────────────────────────────────────────────────────────────────────────────────
📦 brz_redata_balance_balance_electrico
────────────────────────────────────────────────────────────────────────────────

🔹 Estadísticas de 'value':
+-------+--------------------+
|summary|value               |
+-------+--------------------+
|count  |74858               |
|mean   |90730.5399486361    |
|stddev |623420.7797951947   |
|min    |-3106069.988        |
|max    |2.1907194242999997E7|
+-------+--------------------+

🔹 Estadísticas de 'percentage':
+-------+----------------------+
|summary|percentage            |
+-------+----------------------+
|count  |74858                 |
|mean   |0.49231087379325905   |
|min    |1.0194169977452991E-10|
|max    |1.0                   |
+-------+----------------------+

🔹 Top 5 valores máximos:
+-------------------+---------+------------+--------------------+
|datetime           |geo_name |series_title|value               |
+---------

In [7]:
# ============================================
# 4. RECOMENDACIONES PARA SILVER
# ============================================
print("\n" + "=" * 80)
print("💡 4. RECOMENDACIONES PARA CAPA SILVER")
print("=" * 80)

recommendations = """
Basado en el análisis:

1. PARTICIONADO SUGERIDO:
   - Por fecha (year/month) para queries temporales eficientes
   - Por geo_id para análisis regionales

2. TABLAS SILVER PROPUESTAS:
   
   a) slv_demanda_diaria
      - Agregar brz_demanda_evolucion por día
      - Incluir métricas: demanda_real, demanda_prevista
      - Particionar por año/mes y geo_id
   
   b) slv_generacion_mix
      - Combinar estructura_generacion + estructura_renovables
      - Agregar por tecnología y fecha
      - Calcular % renovable vs no renovable
   
   c) slv_balance_horario
      - Desde balance_balance_electrico
      - Mantener granularidad horaria para análisis detallado
      - Enriquecer con datos de precio
   
   d) slv_emisiones
      - Desde estructura_generacion_emisiones_asociadas
      - Agregar por día/mes
      - KPIs: CO2_total, CO2_por_MWh
   
   e) slv_mercado_precios
      - Desde componentes_precio
      - Análisis de componentes del precio eléctrico

3. TRANSFORMACIONES NECESARIAS:
   - Limpieza: Confirmar que no hay duplicados reales
   - Enriquecimiento: Añadir flags (es_renovable, es_peninsula, etc.)
   - Agregaciones: Crear vistas daily/monthly según caso de uso
   - Tipos de datos: Validar que values sean numeric, dates sean timestamp

4. CALIDAD DE DATOS:
   - ✅ Estructura consistente en todas las tablas
   - ✅ Sin nulos críticos en datetime/value
   - ⚠️ Verificar percentage nulls (esperado en algunas series)
   - ⚠️ Confirmar si maxima_renovable tiene suficiente granularidad

5. OPTIMIZACIONES:
   - Z-ORDER por datetime y geo_id en tablas grandes
   - VACUUM periódico para mantener performance
   - Estadísticas de columna para query optimizer
"""

print(recommendations)

print("\n" + "=" * 80)
print("✅ ANÁLISIS COMPLETADO")
print("=" * 80)

StatementMeta(, 9d1c3a59-d18e-4fdf-8498-c7eee8f1135d, 9, Finished, Available, Finished)


💡 4. RECOMENDACIONES PARA CAPA SILVER

Basado en el análisis:

1. PARTICIONADO SUGERIDO:
   - Por fecha (year/month) para queries temporales eficientes
   - Por geo_id para análisis regionales

2. TABLAS SILVER PROPUESTAS:
   
   a) slv_demanda_diaria
      - Agregar brz_demanda_evolucion por día
      - Incluir métricas: demanda_real, demanda_prevista
      - Particionar por año/mes y geo_id
   
   b) slv_generacion_mix
      - Combinar estructura_generacion + estructura_renovables
      - Agregar por tecnología y fecha
      - Calcular % renovable vs no renovable
   
   c) slv_balance_horario
      - Desde balance_balance_electrico
      - Mantener granularidad horaria para análisis detallado
      - Enriquecer con datos de precio
   
   d) slv_emisiones
      - Desde estructura_generacion_emisiones_asociadas
      - Agregar por día/mes
      - KPIs: CO2_total, CO2_por_MWh
   
   e) slv_mercado_precios
      - Desde componentes_precio
      - Análisis de componentes del precio eléct